# Capstone Project: Where should I stay in Tokyo?

## Background & Problem

I am planning a trip to Tokyo and looking for a neighborhood to stay. I hope it would be easy to find local food around, have many café and music clubs where I can linger around in my free time, and have plenty of shops where I can find interesting gifts for my friends.

I read an article about 11 neighborhoods to visit for the first-time traveler and want to choose a neighborhood among them. The neighborhoods are Shinjuku, Shibuya, Akihabara, Harajuku, Ginza, Ueno, Shimokitazawa, Kenji, Nakameguro, Roppongi, and Odaiba. Each is described exciting and attractive to me.

Since the popular kinds of venues in a neighborhood present its style, I will use Foursquare to find out the most popular venues in these 11 neighborhoods and cluster them to see which one most fits my style.

Before exploring it, let's import neccessary libraries.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Tokyo Neighborhood Dataset

#### First I'll get Tokyo's location

In [2]:
address = 'Tokyo'

geolocator = Nominatim(user_agent="tokyo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tokyo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.


#### Location of 11 Neighborhoods in Tokyo

In [3]:
neighborhoods = ['Shinjuku','Shibuya','Akihabara','Harajuku','Ginza','Ueno','Shimokitazawa','Kenji','Nakameguro','Roppongi','Odaiba']
neigh_lat = []
neigh_long = []

for neigh in neighborhoods:
    neigh_location = geolocator.geocode(neigh)
    neigh_lat.append(neigh_location.latitude)
    neigh_long.append(neigh_location.longitude)
    
print(neighborhoods)
print(neigh_lat)
print(neigh_long)

neigh_df = pd.DataFrame(data={'Neighborhood':neighborhoods,'Latitude':neigh_lat,'Longitude':neigh_long})
neigh_df

['Shinjuku', 'Shibuya', 'Akihabara', 'Harajuku', 'Ginza', 'Ueno', 'Shimokitazawa', 'Kenji', 'Nakameguro', 'Roppongi', 'Odaiba']
[35.6937632, 35.6645956, 35.6997364, 35.6687049, 35.66951555, 35.7117587, 35.6597563, 48.1052087, 35.6441395, 35.662819, 35.61912805]
[139.7036319, 139.6987107, 139.7712503, 139.7053357, 139.7643055988, 139.7776445, 139.6688113, -1.6789347, 139.6988317, 139.7313383, 139.779403349221]


,Neighborhood,Latitude,Longitude
0,Shinjuku,35.693763,139.703632
1,Shibuya,35.664596,139.698711
2,Akihabara,35.699736,139.771250
3,Harajuku,35.668705,139.705336
4,Ginza,35.669516,139.764306
5,Ueno,35.711759,139.777645
6,Shimokitazawa,35.659756,139.668811
7,Kenji,48.105209,-1.678935
8,Nakameguro,35.644140,139.698832
9,Roppongi,35.662819,139.731338


#### Create a map of Tokyo with its 11 neighborhoods

In [4]:
# create map of Tokyo using latitude and longitude values
map_tokyo = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(neigh_df['Latitude'], neigh_df['Longitude'], neigh_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tokyo)  

map_tokyo

## Explore the venues around the 11 neighborhoods in Tokyo

In [5]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: LUI4HHCGWQ0SZRKC4UC3FR3GD5RSKDJVBG4KH5SKS4YHYJXP
CLIENT_SECRET:DDSZKMWDL51XBSECTIUCSOO10N3ILQV43OXE3MNIV3WDV0TQ


#### Get the venue data in the radius of 500m around each neighborhood

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
tokyo_venues = getNearbyVenues(names=neigh_df['Neighborhood'],
                                   latitudes=neigh_df['Latitude'],
                                   longitudes=neigh_df['Longitude']
                                  )

Shinjuku
Shibuya
Akihabara
Harajuku
Ginza
Ueno
Shimokitazawa
Kenji
Nakameguro
Roppongi
Odaiba


In [16]:
print('The total number of venues are ',tokyo_venues.shape[0])
tokyo_venues.head()

The total number of venues are  990


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Shinjuku,35.693763,139.703632,Thermae Yu (テルマー湯),35.694526,139.705139,Spa
1,Shinjuku,35.693763,139.703632,Michishirube (道しるべ),35.694744,139.703705,Yakitori Restaurant
2,Shinjuku,35.693763,139.703632,Rock Bar Mother,35.694661,139.702795,Bar
3,Shinjuku,35.693763,139.703632,Metal Bar GODZ,35.694921,139.702918,Bar
4,Shinjuku,35.693763,139.703632,Acacia (アカシア),35.693116,139.701570,Yoshoku Restaurant


In [17]:
tokyo_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Akihabara,100,100,100,100,100,100
Ginza,100,100,100,100,100,100
Harajuku,100,100,100,100,100,100
Kenji,53,53,53,53,53,53
Nakameguro,100,100,100,100,100,100
Odaiba,37,37,37,37,37,37
Roppongi,100,100,100,100,100,100
Shibuya,100,100,100,100,100,100
Shimokitazawa,100,100,100,100,100,100


In [18]:
print('There are {} uniques categories.'.format(len(tokyo_venues['Venue Category'].unique())))

There are 183 uniques categories.


#### Analyze each neighborhood

In [19]:
# one hot encoding
tokyo_onehot = pd.get_dummies(tokyo_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tokyo_onehot['Neighborhood'] = tokyo_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tokyo_onehot.columns[-1]] + list(tokyo_onehot.columns[:-1])
tokyo_onehot = tokyo_onehot[fixed_columns]

tokyo_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,BBQ Joint,Bakery,Bar,Bath House,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Center,Café,Candy Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cultural Center,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Halal Restaurant,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Box,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Men's Store,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Music Store,Music Venue,Nabe Restaurant,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pet Café,Pharmacy,Pier,Pizza Place,Planetarium,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Soba Restaurant,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tonkatsu Restaurant,Toy / Game Store,Trail,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wagashi Place,Whisky Bar,Wine Bar,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,Shinjuku,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Shinjuku,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Shinjuku,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Shinjuku,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [20]:
tokyo_onehot.shape

(990, 184)

In [21]:
tokyo_grouped = tokyo_onehot.groupby('Neighborhood').mean().reset_index()
tokyo_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,BBQ Joint,Bakery,Bar,Bath House,Bed & Breakfast,Beer Bar,Beer Store,Bistro,Bookstore,Boutique,Brazilian Restaurant,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Center,Café,Candy Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cultural Center,Cupcake Shop,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Halal Restaurant,Hawaiian Restaurant,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kaiseki Restaurant,Karaoke Box,Kebab Restaurant,Korean Restaurant,Kushikatsu Restaurant,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Men's Store,Mexican Restaurant,Miscellaneous Shop,Movie Theater,Multiplex,Music Store,Music Venue,Nabe Restaurant,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Pet Café,Pharmacy,Pier,Pizza Place,Planetarium,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Soba Restaurant,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Sukiyaki Restaurant,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Taxi Stand,Tea Room,Tempura Restaurant,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tonkatsu Restaurant,Toy / Game Store,Trail,Train Station,Tree,Tunnel,Udon Restaurant,Unagi Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wagashi Place,Whisky Bar,Wine Bar,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant
0,Akihabara,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.010000,0.01,0.000000,0.00,0.000000,0.00,0.120000,0.00,0.010000,0.00,0.01,0.000000,0.00,0.010000,0.00,0.03,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.00,0.000000,0.04,0.00,0.00,0.13,0.01,0.01,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.01,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.13,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.01,0.020000,0.00,0.00,0.02,0.00,0.01,0.01,0.01,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.02,0.000000,0.000000,0.00,0.000000,0.00,0.01,0.050000,0.00,0.00,0.000000,0.03,0.00,0.00,0.030000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.010000,0.000000,0.01,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.010000,0.000000,0.01,0.00,0.00,0.000000,0.00,0.00,0.00,0.01,0.00,0.00,0.02,0.040000,0.00,0.01,0.00,0.000000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.01
1,Ginza,0.00,0.00,0.00,0.00,0.04,0.00,0.01,0.010000,0.000000,0.00,0.03,0.010000,0.020000,0.00,0.00,0.01,0.00,0.00,0.01,0.04,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.070000,0.01,0.010000,0.03,0.00,0.020000,0.02,0.070000,0.00,0.00,0.010000

In [22]:
tokyo_grouped.shape

(11, 184)

In [23]:
num_top_venues = 5

for hood in tokyo_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tokyo_grouped[tokyo_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akihabara----
                venue  freq
0   Electronics Store  0.13
1          Hobby Shop  0.13
2                Café  0.12
3    Ramen Restaurant  0.05
4  Donburi Restaurant  0.04


----Ginza----
                 venue  freq
0                 Café  0.07
1          Coffee Shop  0.07
2          Art Gallery  0.04
3  Japanese Restaurant  0.04
4             Boutique  0.04


----Harajuku----
            venue  freq
0            Café  0.11
1        Boutique  0.06
2  Clothing Store  0.06
3     Coffee Shop  0.05
4       Gift Shop  0.03


----Kenji----
               venue  freq
0              Hotel  0.15
1  French Restaurant  0.09
2           Creperie  0.08
3   Sushi Restaurant  0.06
4       Cupcake Shop  0.04


----Nakameguro----
                 venue  freq
0  Japanese Restaurant  0.13
1            BBQ Joint  0.06
2          Coffee Shop  0.05
3   Italian Restaurant  0.05
4             Sake Bar  0.04


----Odaiba----
                 venue  freq
0    Convenience Store  0.08
1           H

#### Let's see the top 5 most popular venue categories in each neighborhood

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Put the top 5 popular venues in a dataframe

In [25]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tokyo_grouped['Neighborhood']

for ind in np.arange(tokyo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tokyo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Akihabara,Electronics Store,Hobby Shop,Café,Ramen Restaurant,Arcade
1,Ginza,Coffee Shop,Café,Japanese Restaurant,Boutique,Sushi Restaurant
2,Harajuku,Café,Clothing Store,Boutique,Coffee Shop,Thai Restaurant
3,Kenji,Hotel,French Restaurant,Creperie,Sushi Restaurant,Cupcake Shop
4,Nakameguro,Japanese Restaurant,BBQ Joint,Italian Restaurant,Coffee Shop,Sushi Restaurant
5,Odaiba,Convenience Store,Bus Stop,Intersection,Japanese Restaurant,Hot Spring
6,Roppongi,Japanese Restaurant,Steakhouse,Ramen Restaurant,Chinese Restaurant,Art Gallery
7,Shibuya,Café,Record Shop,Ramen Restaurant,Nightclub,Rock Club
8,Shimokitazawa,Café,Sake Bar,Coffee Shop,Bar,Ramen Restaurant
9,Shinjuku,Bar,Sake Bar,Ramen Restaurant,BBQ Joint,Record Shop


In [26]:
# set number of clusters
kclusters = 3

tokyo_grouped_clustering = tokyo_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tokyo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 2, 0, 0, 0, 1, 1, 0], dtype=int32)

#### Cluster the neighborhoods

In [27]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# change clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

tokyo_merged = neigh_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tokyo_merged = tokyo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tokyo_merged.head() 

,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
0,Shinjuku,35.693763,139.703632,Bar,Sake Bar,Ramen Restaurant,BBQ Joint,Record Shop,0
1,Shibuya,35.664596,139.698711,Café,Record Shop,Ramen Restaurant,Nightclub,Rock Club,1
2,Akihabara,35.699736,139.771250,Electronics Store,Hobby Shop,Café,Ramen Restaurant,Arcade,1
3,Harajuku,35.668705,139.705336,Café,Clothing Store,Boutique,Coffee Shop,Thai Restaurant,1
4,Ginza,35.669516,139.764306,Coffee Shop,Café,Japanese Restaurant,Boutique,Sushi Restaurant,1


In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)] #why?
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tokyo_merged['Latitude'], tokyo_merged['Longitude'], tokyo_merged['Neighborhood'], tokyo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Visualize the clusters in Tokyo map**

In [29]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 0, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
0,Shinjuku,Bar,Sake Bar,Ramen Restaurant,BBQ Joint,Record Shop,0
5,Ueno,Sake Bar,Japanese Restaurant,Ramen Restaurant,Chinese Restaurant,Bed & Breakfast,0
8,Nakameguro,Japanese Restaurant,BBQ Joint,Italian Restaurant,Coffee Shop,Sushi Restaurant,0
9,Roppongi,Japanese Restaurant,Steakhouse,Ramen Restaurant,Chinese Restaurant,Art Gallery,0
10,Odaiba,Convenience Store,Bus Stop,Intersection,Japanese Restaurant,Hot Spring,0


#### Explore each cluster

In [30]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 1, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
1,Shibuya,Café,Record Shop,Ramen Restaurant,Nightclub,Rock Club,1
2,Akihabara,Electronics Store,Hobby Shop,Café,Ramen Restaurant,Arcade,1
3,Harajuku,Café,Clothing Store,Boutique,Coffee Shop,Thai Restaurant,1
4,Ginza,Coffee Shop,Café,Japanese Restaurant,Boutique,Sushi Restaurant,1
6,Shimokitazawa,Café,Sake Bar,Coffee Shop,Bar,Ramen Restaurant,1


In [33]:
tokyo_merged.loc[tokyo_merged['Cluster Labels'] == 2, tokyo_merged.columns[[0] + list(range(3, tokyo_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Cluster Labels
7,Kenji,Hotel,French Restaurant,Creperie,Sushi Restaurant,Cupcake Shop,2


So the first cluster is about restaurants and bars, while the second cluster has many coffee shops, restaurants, and stores where I can find gifts, and the third cluster has many hotels and restaurants.

## Conclusion

**The second cluster including Shibuya, Akihabara, Harajuku, Ginza and  seems to fit my style the most, so I will book a place in these neighborhoods.**